In [47]:
import astropy
import numpy as np
from astropy.table import Table

import sys
sys.path.insert(1, '/Users/nityaravi/Documents/GitHub/RotationCurves/')

from spirals.rotation_curve_functions import disk_mass
from astropy.io import fits

In [3]:
NSA_FN = '/Users/nityaravi/Documents/Research/RotationCurves/data/nsa_v1_0_1.fits'
DISK_FITS_FN = '/Users/nityaravi/Documents/Research/RotationCurves/data/manga/output_files/DR17/Pipe3d_disk_bulge.fits'

In [4]:
galaxies_table = Table.read(DISK_FITS_FN, format='fits')
nsa_table = Table.read(NSA_FN, format='fits')

In [5]:
len(galaxies_table)

11273

In [7]:
# indices of galaxies in nsa_table
nsa_index = np.ones(len(galaxies_table))*np.nan
nsa_ids = galaxies_table['nsa_nsaid']
for i in range(0, len(galaxies_table)):
    
    if nsa_ids[i] >= 0:
        nsa_index[i] = np.where(nsa_table['NSAID'] == nsa_ids[i])[0][0]
        
    if i % 1000 ==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [12]:
nsa_index

array([ 48157.,  48188.,  55648., ..., 294374., 294459., 550782.])

In [13]:
R90 = np.ones(len(nsa_index))*np.nan
for i in range(0, len(nsa_index)):
    if np.isfinite(nsa_index[i]):
        R90[i] = nsa_table['ELPETRO_TH90_R'][int(nsa_index[i])]
    
    if i % 1000 ==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [52]:
z = galaxies_table['nsa_z']
H_0 = 100      # Hubble's Constant in units of h km/s/Mpc
c = 299792.458 # Speed of light in units of km/s

In [34]:
# add R90 to galaxies_table
galaxies_table['nsa_elpetro_th90'] = R90

In [53]:
dist_to_galaxy_Mpc = c*z/H_0
dist_to_galaxy_kpc = dist_to_galaxy_Mpc*1000

R90_kpc = dist_to_galaxy_kpc*np.tan(R90*(1./60)*(1./60)*(np.pi/180))

In [54]:
function_parameters = {'Sigma_disk': galaxies_table['Sigma_disk'],
                   'R_disk': galaxies_table['R_disk'],
                   'R_bulge': galaxies_table['R_bulge'],
                   'rho_bulge': galaxies_table['rho_bulge'],
                   'Sigma_disk_err': galaxies_table['Sigma_disk_err'],
                   'R_disk_err': galaxies_table['R_disk_err'],
                   'R_bulge_err': galaxies_table['R_bulge_err'],
                   'rho_bulge_err': galaxies_table['rho_bulge_err'],
}

In [55]:
M90_disk, M90_disk_err = disk_mass(function_parameters, R90_kpc)
M_disk, M_disk_err = disk_mass(function_parameters, 3.5*R90_kpc)

/Users/nityaravi/Documents/GitHub/RotationCurves/spirals/rotation_curve_functions.py:277: RuntimeWarning: overflow encountered in exp
  Mdisk = 2*np.pi*SigD*Rd*(Rd - np.exp(-r/Rd)*(r + Rd))
/Users/nityaravi/Documents/GitHub/RotationCurves/spirals/rotation_curve_functions.py:279: RuntimeWarning: overflow encountered in exp
  Mdisk_err2 = (4*np.pi*Rd*SigD - 2*np.exp(-r/Rd)*np.pi*SigD*(r**2 + 2*r*Rd +2*Rd**2)/Rd)**2\
/Users/nityaravi/Documents/GitHub/RotationCurves/spirals/rotation_curve_functions.py:288: RuntimeWarning: overflow encountered in exp
  F = 1 - np.exp(-x) * (1 + x + x**2 / 2)
/Users/nityaravi/Documents/GitHub/RotationCurves/spirals/rotation_curve_functions.py:291: RuntimeWarning: overflow encountered in exp
  M_bulge_err2= ((4*np.pi*rho_bulge/R_bulge)*(6*R_bulge**3-np.exp(-x) \
/Users/nityaravi/Documents/GitHub/RotationCurves/spirals/rotation_curve_functions.py:300: RuntimeWarning: divide by zero encountered in log10
  return np.log10(Mdisk_tot), np.log10(Mdisk_tot_err)


In [56]:
galaxies_table['M90_disk'] = M90_disk
galaxies_table['M90_disk_err'] = M90_disk_err
galaxies_table['M_disk'] = M_disk
galaxies_table['M_disk_err'] = M_disk_err

In [57]:
galaxies_table.write('/Users/nityaravi/Documents/Research/RotationCurves/data/manga/output_files/DR17/disk_masses.fits',
                      format='fits', overwrite=True)